In [1]:
%load_ext autoreload
%autoreload 2

import itertools
import numpy as np
import matplotlib.pyplot as plt

from multiprocessing import Pool
import time
from tqdm import tqdm

from graph_generation import create_ER, dataset_nclass_ER
from models import GCN, GCNEmbed
from train import Trainer, Dataset
from embedding import Embedding
from experiments import experiment_embedding

import torch
from torch_geometric.loader import DataLoader
device = torch.device('cuda')

from scipy import stats

import yaml

In [2]:
config = yaml.safe_load(open("configs.yml"))

In [3]:
dataset_grafi_nx, dataset_labels, list_p= dataset_nclass_ER(config)

Mean connectivity for each node: 14.964 p=0.05
Mean connectivity for each node: 89.701 p=0.3
Labels values: {0.05, 0.3}


In [ ]:
embeddings, trainer = experiment_embedding(config, dataset_grafi_nx, dataset_labels, list_p)

In [ ]:
plt.figure(figsize=(18, 6)) #, dpi=60)
for p in list_p:
    mask_int = np.argwhere(embeddings.embedding_labels==p).flatten()
    emb = embeddings.embeddings_array[mask_int].flatten()
    
    h,e = np.histogram(emb, bins=10, density=True)
    x = np.linspace(e.min(), e.max())
    plt.bar(e[:-1], h, width=np.diff(e), ec='k', align='edge', label='histogram')

    kde = stats.gaussian_kde(emb)
    plt.plot(x, kde.pdf(x),  lw=5, label='KDE')
    
plt.xlabel('p', fontsize=18)
plt.xticks(fontsize=18) 
plt.show()